In [1]:
import requests
import pandas as pd
import json
import os 
from dotenv import load_dotenv
import joblib
import sqlite3
import time

In [2]:
load_dotenv()

GECKO_API = os.getenv("GECKO_API")
if not GECKO_API:
    raise ValueError("GECKO_API environment variable is not set.")

In [3]:
network = "eth"
token_address = "0xdAC17F958D2ee523a2206206994597C13D831ec7"  # USDT


# Format URL properly
url = f"https://pro-api.coingecko.com/api/v3/onchain/networks/{network}/tokens/{token_address}/pools"

# Get API key from environment variable
API_KEY = os.getenv("GECKO_API")
if not API_KEY:
    raise ValueError("GECKO_API environment variable not set.")

headers = {
    "accept": "application/json",
    "x-cg-pro-api-key": API_KEY
}

# Make request
response = requests.get(url, headers=headers)
response.raise_for_status()
data = response.json()

In [4]:
# All USDT pair pools in the ethereum blockchain

all_pools = []

for pool in data["data"]:
    
    pool_data = {
        "id": pool["id"],
        "name": pool["attributes"]["name"],
        "pool_created_at": pool["attributes"]["pool_created_at"],
        "address": pool["attributes"]["address"]
    }
    all_pools.append(pool_data)

# Convert to DataFrame
df_pools = pd.DataFrame(all_pools)

df_pools.tail()

,id,name,pool_created_at,address
15,eth_0xce93ea3914c62e0008348cf39fd006e130e7c503...,USDe / USDT 0.001%,2025-08-26T11:21:59Z,0xce93ea3914c62e0008348cf39fd006e130e7c503935f...
16,eth_0x6ca298d2983ab03aa1da7679389d955a4efee15c,WETH / USDT 0.05%,2023-04-02T04:55:54Z,0x6ca298d2983ab03aa1da7679389d955a4efee15c
17,eth_0x31373595f40ea48a7aab6cbcb0d377c6066e2dca,USDC / USDT 0.001%,2025-02-28T11:01:17Z,0x31373595f40ea48a7aab6cbcb0d377c6066e2dca
18,eth_0x48da0965ab2d2cbf1c17c09cfb5cbe67ad5b1406,DAI / USDT 0.01%,2022-07-14T19:47:16Z,0x48da0965ab2d2cbf1c17c09cfb5cbe67ad5b1406
19,eth_0x390f3595bca2df7d23783dfd126427cceb997bf4,crvUSD / USDT,2023-05-18T16:16:53Z,0x390f3595bca2df7d23783dfd126427cceb997bf4


In [5]:
# Converting pool created at to datetime
# For easier calculations, sorting and resampling or grouping by time

# Convert column to datetime (UTC)
df_pools['pool_created_at'] = pd.to_datetime(df_pools['pool_created_at'], utc=True)

# Creating a new column for pool age in days
df_pools['pool_age_days'] = (pd.Timestamp.now(tz='UTC') - df_pools['pool_created_at']).dt.days

df_pools.head()

,id,name,pool_created_at,address,pool_age_days
0,eth_0x8aa4e11cbdf30eedc92100f4c8a31ff748e201d4...,USDC / USDT 0.001%,2025-04-01 02:10:47+00:00,0x8aa4e11cbdf30eedc92100f4c8a31ff748e201d44712...,158
1,eth_0x4e68ccd3e89f51c3074ca5072bbac773960dfa36,WETH / USDT 0.3%,2021-12-29 12:36:12+00:00,0x4e68ccd3e89f51c3074ca5072bbac773960dfa36,1347
2,eth_0x00836fe54625be242bcfa286207795405ca4fd10,sUSDS / USDT,2025-04-18 22:52:10+00:00,0x00836fe54625be242bcfa286207795405ca4fd10,140
3,eth_0xaae9da4a878406eb1de54efac30e239fd56d54fb...,USDe / USDT 0.006%,2025-08-04 13:21:23+00:00,0xaae9da4a878406eb1de54efac30e239fd56d54fb8051...,33
4,eth_0x11b815efb8f581194ae79006d24e0d814b7697f6,WETH / USDT 0.05%,2021-12-29 12:35:27+00:00,0x11b815efb8f581194ae79006d24e0d814b7697f6,1347


In [6]:
# Extract fee into its own column (keep the % sign)

df_pools["fee"] = df_pools["name"].str.extract(r"(\d+\.?\d*%)")

# Remove the fee part from the name
df_pools["name"] = df_pools["name"].str.replace(r"\s\d+(\.\d+)?%", "", regex=True)

df_pools.head()

,id,name,pool_created_at,address,pool_age_days,fee
0,eth_0x8aa4e11cbdf30eedc92100f4c8a31ff748e201d4...,USDC / USDT,2025-04-01 02:10:47+00:00,0x8aa4e11cbdf30eedc92100f4c8a31ff748e201d44712...,158,0.001%
1,eth_0x4e68ccd3e89f51c3074ca5072bbac773960dfa36,WETH / USDT,2021-12-29 12:36:12+00:00,0x4e68ccd3e89f51c3074ca5072bbac773960dfa36,1347,0.3%
2,eth_0x00836fe54625be242bcfa286207795405ca4fd10,sUSDS / USDT,2025-04-18 22:52:10+00:00,0x00836fe54625be242bcfa286207795405ca4fd10,140,NaN
3,eth_0xaae9da4a878406eb1de54efac30e239fd56d54fb...,USDe / USDT,2025-08-04 13:21:23+00:00,0xaae9da4a878406eb1de54efac30e239fd56d54fb8051...,33,0.006%
4,eth_0x11b815efb8f581194ae79006d24e0d814b7697f6,WETH / USDT,2021-12-29 12:35:27+00:00,0x11b815efb8f581194ae79006d24e0d814b7697f6,1347,0.05%


In [7]:
#  The joblib directory i want to save extracted pools data to

path = r'/home/realist/projects/DexTracker/backend/Database'
os.makedirs(path, exist_ok=True)

In [8]:
# Save pools dataframe as .pkl
joblib.dump(df_pools, os.path.join(path, "USDT-pairs_pools.pkl"))

['/home/realist/projects/DexTracker/backend/Database/USDT-pairs_pools.pkl']

In [9]:
# Not calling for the quote token since its a stablecoin
# base_token_price_usd → For universal USD pricing
# base_token_price_quote_token → For the actual trading rate in the pool

# All USDT pair price in the ethereum blockchain

pair_price = []

for price in data["data"]:
    
    token_data = {
        "name": price["attributes"]["name"],
        "base_price_usd": price["attributes"]["base_token_price_usd"],
        "base_quote_price": price["attributes"]["base_token_price_quote_token"]
    }
    pair_price.append(token_data)

# Convert to DataFrame
df_prices = pd.DataFrame(pair_price)

df_prices.tail()

,name,base_price_usd,base_quote_price
15,USDe / USDT 0.001%,1.00336594888672,1.0037764333
16,WETH / USDT 0.05%,4289.07224267424,4275.884116
17,USDC / USDT 0.001%,1.00008055561787,0.9971563211
18,DAI / USDT 0.01%,0.996904740576491,0.993837841
19,crvUSD / USDT,0.991904899493134,1.0035779035


In [10]:
# Save pool price dataframe as .pkl
joblib.dump(df_prices, os.path.join(path, "USDT-pairs_prices.pkl"))

['/home/realist/projects/DexTracker/backend/Database/USDT-pairs_prices.pkl']

In [11]:
# TVL (Liquidity for USDT pairs)

pair_tvl = []

for liquidity in data["data"]:
    
    pair_liquidity = {
        "name": liquidity["attributes"]["name"],
        "pair_reserve_in_usd": liquidity["attributes"]["reserve_in_usd"]
    }
    pair_tvl.append(pair_liquidity)

# Convert to DataFrame
df_tvl = pd.DataFrame(pair_tvl)

df_tvl.tail()

,name,pair_reserve_in_usd
15,USDe / USDT 0.001%,2724776.0811
16,WETH / USDT 0.05%,16491507.6497
17,USDC / USDT 0.001%,2121723.5035
18,DAI / USDT 0.01%,7072395.3338
19,crvUSD / USDT,9776172.5187


In [12]:
# Save pools tvl dataframe as .pkl
joblib.dump(df_tvl, os.path.join(path, "USDT-pairs_tvl.pkl"))

['/home/realist/projects/DexTracker/backend/Database/USDT-pairs_tvl.pkl']

In [13]:
# Market metrics (FDV for USDT pairs)

pair_fdv = []

for diluted_value in data["data"]:
    
    pair_fully_diluted_value = {
        "name": diluted_value["attributes"]["name"],
        "fdv_usd": diluted_value["attributes"]["fdv_usd"]
    }
    pair_fdv.append(pair_fully_diluted_value)

# Convert to DataFrame
df_fdv = pd.DataFrame(pair_fdv)

df_fdv.tail()

,name,fdv_usd
15,USDe / USDT 0.001%,87003992214.6638
16,WETH / USDT 0.05%,86904878746.3773
17,USDC / USDT 0.001%,87039428250.3213
18,DAI / USDT 0.01%,87052735997.241
19,crvUSD / USDT,86089670962.4132


In [14]:
# Save pools fdv dataframe as .pkl
joblib.dump(df_fdv, os.path.join(path, "USDT-pairs_fdv.pkl"))

['/home/realist/projects/DexTracker/backend/Database/USDT-pairs_fdv.pkl']

In [15]:
# Price change

pair_price_change = []

for change in data["data"]:
    attr = change["attributes"]
    price_change = attr.get("price_change_percentage", {})  # This gets the dictionary

    # Create a dictionary combining pool name and price changes
    pool_price_change = {
        "name": attr["name"],
        "m5": float(price_change.get("m5", 0)),
        "m15": float(price_change.get("m15", 0)),
        "m30": float(price_change.get("m30", 0)),
        "h1": float(price_change.get("h1", 0)),
        "h6": float(price_change.get("h6", 0)),
        "h24": float(price_change.get("h24", 0))
    }
    pair_price_change.append(pool_price_change)

# Convert to DataFrame
df_price_chg_pct = pd.DataFrame(pair_price_change)

df_price_chg_pct.tail()

,name,m5,m15,m30,h1,h6,h24
15,USDe / USDT 0.001%,0.00,-0.07,-0.14,0.29,0.73,0.00
16,WETH / USDT 0.05%,0.06,-0.05,0.18,-0.12,-0.07,0.19
17,USDC / USDT 0.001%,0.00,0.60,0.58,-0.05,0.57,0.04
18,DAI / USDT 0.01%,0.00,0.00,0.44,0.00,0.61,0.61
19,crvUSD / USDT,0.00,0.08,0.17,-0.24,0.36,-0.05


In [16]:
# Save pools percentage change in price dataframe as .pkl
joblib.dump(df_price_chg_pct, os.path.join(path, "USDT-pairs_price_chg_pct.pkl"))

['/home/realist/projects/DexTracker/backend/Database/USDT-pairs_price_chg_pct.pkl']

In [17]:
pair_transactions = []

for tnxs in data["data"]:
    attr = tnxs["attributes"]
    tx = attr.get("transactions", {})

    pair_tx = {
        "name": attr["name"],
        "buys_5m": tx.get("m5", {}).get("buys", 0),
        "sells_5m": tx.get("m5", {}).get("sells", 0),
        "buyers_5m": tx.get("m5", {}).get("buyers", 0),
        "sellers_5m": tx.get("m5", {}).get("sellers", 0),
        "buys_15m": tx.get("m15", {}).get("buys", 0),
        "sells_15m": tx.get("m15", {}).get("sells", 0),
        "buyers_15m": tx.get("m15", {}).get("buyers", 0),
        "sellers_15m": tx.get("m15", {}).get("sellers", 0),
        "buys_1h": tx.get("h1", {}).get("buys", 0),
        "sells_1h": tx.get("h1", {}).get("sells", 0),
        "buyers_1h": tx.get("h1", {}).get("buyers", 0),
        "sellers_1h": tx.get("h1", {}).get("sellers", 0),
        "buys_24h": tx.get("h24", {}).get("buys", 0),
        "sells_24h": tx.get("h24", {}).get("sells", 0),
        "buyers_24h": tx.get("h24", {}).get("buyers", 0),
        "sellers_24h": tx.get("h24", {}).get("sellers", 0),
    }

    pair_transactions.append(pair_tx)

# Convert to DataFrame
df_transactions = pd.DataFrame(pair_transactions)

df_transactions.tail()

,name,buys_5m,sells_5m,buyers_5m,sellers_5m,buys_15m,sells_15m,buyers_15m,sellers_15m,buys_1h,sells_1h,buyers_1h,sellers_1h,buys_24h,sells_24h,buyers_24h,sellers_24h
15,USDe / USDT 0.001%,0,1,0.0,1.0,0,3,0.0,2.0,3,7,3.0,6.0,74,82,43.0,61.0
16,WETH / USDT 0.05%,7,0,7.0,0.0,8,4,8.0,4.0,50,43,44.0,35.0,510,450,320.0,192.0
17,USDC / USDT 0.001%,0,1,0.0,1.0,1,1,1.0,1.0,12,4,3.0,4.0,196,214,82.0,89.0
18,DAI / USDT 0.01%,4,0,4.0,0.0,5,0,5.0,0.0,16,2,14.0,2.0,128,104,106.0,56.0
19,crvUSD / USDT,0,1,0.0,1.0,2,2,2.0,2.0,14,7,9.0,5.0,143,97,23.0,19.0


In [18]:
# Save pools transaction activity dataframe as .pkl

joblib.dump(df_transactions, os.path.join(path, "USDT-pairs_transactions.pkl"))

['/home/realist/projects/DexTracker/backend/Database/USDT-pairs_transactions.pkl']

In [19]:
pair_volume = []

for volumes in data["data"]:
    attr = volumes["attributes"]
    vol = attr.get("volume_usd", {})

    pool_vol = {
        "name": attr["name"],
        "vol_5m": float(vol.get("m5", 0)),
        "vol_15m": float(vol.get("m15", 0)),
        "vol_30m": float(vol.get("m30", 0)),
        "vol_1h": float(vol.get("h1", 0)),
        "vol_6h": float(vol.get("h6", 0)),
        "vol_24h": float(vol.get("h24", 0))
    }

    pair_volume.append(pool_vol)

# Convert to DataFrame
df_volume = pd.DataFrame(pair_volume)

df_volume.tail()

,name,vol_5m,vol_15m,vol_30m,vol_1h,vol_6h,vol_24h
15,USDe / USDT 0.001%,11986.584262,224620.459980,237467.652007,689106.247946,3.457176e+06,1.292107e+07
16,WETH / USDT 0.05%,5460.295633,14017.659069,85469.221053,459348.406266,7.626757e+05,1.869089e+06
17,USDC / USDT 0.001%,1415.352407,1418.116913,554039.436154,943390.927433,4.224655e+06,1.310556e+07
18,DAI / USDT 0.01%,7670.533519,7859.700995,38115.771232,42287.848130,1.772743e+06,3.557165e+06
19,crvUSD / USDT,893.861494,59437.850642,85238.657215,124910.767208,6.650379e+05,2.017433e+06


In [20]:
# Save pools transaction activity dataframe as .pkl

joblib.dump(df_volume, os.path.join(path, "USDT-pairs_volume.pkl"))

['/home/realist/projects/DexTracker/backend/Database/USDT-pairs_volume.pkl']

In [21]:
pair_dex = []

for dex_pool in data["data"]:
    dex_info = dex_pool.get("relationships", {}).get("dex", {}).get("data", {})
    pool_dex = {
        "name": pool["attributes"]["name"],
        "dex": dex_info.get("id", "unknown")  # fallback if missing
    }
    pair_dex.append(pool_dex)

df_dex = pd.DataFrame(pair_dex)

df_dex["name"] = df_dex["name"].str.replace(r"\s\d+(\.\d+)?%", "", regex=True)


df_dex.tail()

,name,dex
15,crvUSD / USDT,uniswap-v4-ethereum
16,crvUSD / USDT,pancakeswap-v3-ethereum
17,crvUSD / USDT,maverick-v2-eth
18,crvUSD / USDT,uniswap_v3
19,crvUSD / USDT,curve


In [22]:
# Save pools transaction activity dataframe as .pkl

joblib.dump(df_dex, os.path.join(path, "USDT-pairs_dex.pkl"))

['/home/realist/projects/DexTracker/backend/Database/USDT-pairs_dex.pkl']